# Napari Visualization Pipeline

Note: This pipeline assumes that all microscopy images are already grayscale.

In [2]:
import napari
import numpy as np
from skimage import morphology, restoration, filters, measure
from tifffile import imread

#img = imread('cell-data/parent_control_z3_ch00.tif') --> This was an initial testing image

# Current testing images
#nuclei_img = imread('cell-data/test-image-set/22KO_mCHC17_GFP_3_z3_ch00.tif')
#transfection_img = imread('cell-data/test-image-set/22KO_mCHC17_GFP_3_z3_ch01.tif')
#p115_img = imread('cell-data/test-image-set/22KO_mCHC17_GFP_3_z3_ch02.tif')

nuclei_img = imread('cell-data/test-image-set-2/parent_mCHC17_GFP_4_z3_ch00.tif')
transfection_img = imread('cell-data/test-image-set-2/parent_mCHC17_GFP_4_z2_ch01.tif')
p115_img = imread('cell-data/test-image-set-2/parent_mCHC17_GFP_4_z2_ch02.tif')

viewer = napari.Viewer()
viewer.add_image(nuclei_img, name='nuclei')
viewer.theme = 'light'

### Erosion

In [4]:
from skimage.morphology import erosion, disk

eroded = erosion(nuclei_img, disk(1)) # Change to 2 or 3 for more intense erosion

viewer.add_image(eroded, name='eroded')

<Image layer 'eroded' at 0x3232482c0>

### Estimate noise and denoise via non-local means

In [5]:
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.util import img_as_float

img_float = img_as_float(eroded)

sigma_est = np.mean(estimate_sigma(img_float, channel_axis=None)) # Estimated noise standard deviation. Lower SD means lower estimated noise.

denoised = denoise_nl_means(
    img_float,
    h=0.8 * sigma_est, # h is filter strength. This line sets filter strength to 80% of estimated noise level, but tweak this number if necessary.
    patch_size=11, # Larger value = captures more context; better denoising but loses fine detail (probably doesn't matter for this use).
    patch_distance=12, # Larger value = searches wider area, so better denoising but slower processing.
    fast_mode=True # Make False for more accurate denoising (but it'll be a bit slower).
)

viewer.add_image(denoised, name='denoised')

<Image layer 'denoised' at 0x321f0b680>

### Create nucleus mask

In [6]:
threshold = 0.05 # Play around with the threshold to find the optimal value.
thresholded = denoised > threshold # For each pixel --> if it's brighter than the threshold value, count it as foreground (part of a nucleus).
# The line above creates a boolean array.

viewer.add_labels(thresholded.astype(int), name='thresholded') # This turns the boolean array into binaries.

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


<Labels layer 'thresholded' at 0x32b8c4470>

### Fill any gaps in the mask regions

In [7]:
from scipy.ndimage import binary_fill_holes
import numpy as np

# Assuming 'mask' is a binary numpy array where nuclei are True (1) and background is False (0)
filled_mask = binary_fill_holes(thresholded)

# Convert back to int if needed
filled_mask = filled_mask.astype(np.uint8)

viewer.add_labels(filled_mask, name='hole-filled-mask')

<Labels layer 'hole-filled-mask' at 0x33510fc20>

### Separate mask into distinct regions with integer labels

In [8]:
from skimage.measure import label

labeled_mask = label(filled_mask) # Label connected regions of the mask

viewer.add_labels(labeled_mask, name='nuclei_mask')

<Labels layer 'nuclei_mask' at 0x335188110>

### Clean mask by removing nucleus regions that are cut off by the image and by removing isolated pixels that remain after erosion and denoising

In [9]:
from skimage.measure import regionprops
import numpy as np

image_height, image_width = labeled_mask.shape
cleaned_mask = np.zeros_like(labeled_mask)

for region in regionprops(labeled_mask):
    min_row, min_column, max_row, max_column = region.bbox

    # This checks for labeled regions at the edges of the image
    touches_edge = (min_row == 0 or min_column == 0 or max_row == image_height or max_column == image_width)
    
    if not touches_edge and region.area >= 500:
        cleaned_mask[labeled_mask == region.label] = region.label 
        # This line sets all pixels in cleaned_mask at positions where the boolean mask is True to the integer value region.label.
        # Each specific integer value is a specific label/color.
        
viewer.add_labels(cleaned_mask, name='cleaned_mask')

<Labels layer 'cleaned_mask' at 0x335097c20>

### Determine the image's microns per pixel ratio and create the first donut mask --> PERSONAL NOTE: Confirm the output with Fiji since this is a new image

In [10]:
from skimage.morphology import dilation, disk
from skimage.segmentation import find_boundaries
import tifffile

with tifffile.TiffFile('cell-data/test-image-set/22KO_mCHC17_GFP_3_z3_ch00.tif') as tif:
    page = tif.pages[0]
    tags = page.tags

    x_res = tags['XResolution'].value  # (num, denom)
    y_res = tags['YResolution'].value
    res_unit = tags['ResolutionUnit'].value  # 3 = centimeters

    # Convert to pixels per micrometer
    if res_unit == 3:  # Centimeters
        x_ppcm = x_res[0] / x_res[1]
        y_ppcm = y_res[0] / y_res[1]
        x_ppum = x_ppcm / 10000  # cm → µm
        y_ppum = y_ppcm / 10000
        print('Compare these values to the metadata in Fiji ImageJ:')
        print(f'X pixels/µm: {x_ppum:.4f}')
        print(f'Y pixels/µm: {y_ppum:.4f}')
    else:
        print('Unsupported resolution unit:', res_unit)

microns_per_pixel = (1 / x_ppum + 1 / y_ppum) / 2
desired_donut_width_um = 4.5
n_pixels = int(desired_donut_width_um / microns_per_pixel)

outer_mask = dilation(cleaned_mask, disk(n_pixels))

donut_mask = outer_mask - cleaned_mask
donut_mask[donut_mask < 0] = 0  # Just in case

donut_mask = donut_mask.astype(np.uint8)

viewer.add_labels(donut_mask, name='donut')

Compare these values to the metadata in Fiji ImageJ:
X pixels/µm: 5.5440
Y pixels/µm: 5.5440


<Labels layer 'donut' at 0x3286829c0>

### Create the outer donut mask

In [11]:
desired_outer_width_um = 1.5
n_pixels = int(desired_outer_width_um / microns_per_pixel)

outer_donut_mask = np.zeros_like(cleaned_mask, dtype=np.uint16)

regions = regionprops(cleaned_mask)

for region in regions:
    nucleus_mask = (cleaned_mask == region.label)
    first_donut_mask = (donut_mask == region.label)
    excluded_area = nucleus_mask | first_donut_mask
    outer_mask = dilation(excluded_area, disk(n_pixels))
    outer_donut = outer_mask & (~excluded_area)
    outer_donut_mask[outer_donut] = region.label

viewer.add_labels(outer_donut_mask, name='outer_donut')

<Labels layer 'outer_donut' at 0x3350e9c70>

### **Initial overlapping method (more extreme)**: Delete all overlapping regions --> REVIEW THIS CODE ENTIRELY TO FULLY UNDERSTAND IT

In [89]:
from skimage.morphology import square

# Step 1: Get all unique labels
labels = np.unique(cleaned_mask)
labels = labels[labels != 0]

# Step 2: Build combined masks per region
regions = []
for label_id in labels:
    combined_mask = (
        (cleaned_mask == label_id) |
        (donut_mask == label_id) |
        (outer_donut_mask == label_id)
    )
    regions.append({
        'label': label_id,
        'mask': combined_mask
    })

# Step 3: Find touching regions using dilation
labels_to_remove = set()

for i, region_a in enumerate(regions):
    dilated_a = dilation(region_a['mask'], square(3))  # 8-connected dilation

    for j, region_b in enumerate(regions):
        if i >= j:
            continue  # avoid duplicate checks

        if np.any(dilated_a & region_b['mask']):
            labels_to_remove.add(region_a['label'])
            labels_to_remove.add(region_b['label'])

# Step 4: Remove touching labels from all masks
def remove_labels(mask, labels_to_remove):
    output = np.copy(mask)
    output[np.isin(mask, list(labels_to_remove))] = 0
    return output.astype(np.int32)

cleaned_filtered = remove_labels(cleaned_mask, labels_to_remove)
donut_filtered = remove_labels(donut_mask, labels_to_remove)
outer_donut_filtered = remove_labels(outer_donut_mask, labels_to_remove)

viewer.add_labels(cleaned_filtered, name='no_overlap_nuclei')
viewer.add_labels(donut_filtered, name='no_overlap_donuts')
viewer.add_labels(outer_donut_filtered, name='no_overlap_outer_donut')

/var/folders/zc/r5_5f1x56sq9gtmyssh_0g300000gn/T/ipykernel_39003/2779773151.py:24: FutureWarning: `square` is deprecated since version 0.25 and will be removed in version 0.27. Use `skimage.morphology.footprint_rectangle` instead.
  dilated_a = dilation(region_a['mask'], square(3))  # 8-connected dilation


<Labels layer 'no_overlap_outer_donut' at 0x32fb454f0>

### **Alternative overlapping method (more lenient)**: Remove regions whose outer donuts overlap with other regions' inner donuts and vice versa --> REVIEW THIS CODE ENTIRELY TO FULLY UNDERSTAND IT

In [12]:
from skimage.morphology import binary_dilation, square

labels = np.unique(cleaned_mask)
labels = labels[labels != 0]

regions = []
for label_id in labels:
    region = {
        'label': label_id,
        'nucleus': (cleaned_mask == label_id),
        'donut': (donut_mask == label_id),
        'outer': (outer_donut_mask == label_id)
    }
    regions.append(region)

labels_to_remove = set()

for i, region_a in enumerate(regions):
    inner_donut_a = region_a['donut']
    outer_donut_a = region_a['outer']

    for j, region_b in enumerate(regions):
        if i == j:
            continue

        inner_donut_b = region_b['donut']
        outer_donut_b = region_b['outer']

        # Check for inner donut of A overlapping outer donut of B
        if np.any(inner_donut_a & outer_donut_b):
            labels_to_remove.add(region_a['label'])
            labels_to_remove.add(region_b['label'])

        # Check for outer donut of A overlapping inner donut of B
        elif np.any(outer_donut_a & inner_donut_b):
            labels_to_remove.add(region_a['label'])
            labels_to_remove.add(region_b['label'])

# Remove only labels that meet this specific donut overlap condition
def remove_labels(mask, labels_to_remove):
    output = np.copy(mask)
    output[np.isin(mask, list(labels_to_remove))] = 0
    return output.astype(np.int32)

cleaned_filtered = remove_labels(cleaned_mask, labels_to_remove)
donut_filtered = remove_labels(donut_mask, labels_to_remove)
outer_donut_filtered = remove_labels(outer_donut_mask, labels_to_remove)

viewer.add_labels(cleaned_filtered, name='filtered_nuclei')
viewer.add_labels(donut_filtered, name='filtered_donuts')
viewer.add_labels(outer_donut_filtered, name='filtered_outer_donuts')

<Labels layer 'filtered_outer_donuts' at 0x32876d820>

### Delete regions with cut-off donuts

In [13]:
image_height, image_width = cleaned_filtered.shape
pre_tf_nuclei = np.zeros_like(cleaned_filtered)
pre_tf_donuts = np.zeros_like(donut_filtered)
pre_tf_outer_donuts = np.zeros_like(outer_donut_filtered)

for region in regions: # Using the list from the previous cell
    label_id = region['label']
    
    combined = (
        (cleaned_filtered == label_id) |
        (donut_filtered == label_id) |
        (outer_donut_filtered == label_id)
    )

    coords = np.argwhere(combined)
    if coords.size == 0:
        continue
    min_row, min_col = coords.min(axis=0)
    max_row, max_col = coords.max(axis=0)

    touches_edge = (min_row == 0 or min_col == 0 or max_row == image_height or max_col == image_width)

    if not touches_edge:
        pre_tf_nuclei[cleaned_filtered == label_id] = label_id
        pre_tf_donuts[donut_filtered == label_id] = label_id
        pre_tf_outer_donuts[outer_donut_filtered == label_id] = label_id

viewer.add_labels(pre_tf_nuclei, name='pre_transfection_nuclei')
viewer.add_labels(pre_tf_donuts, name='pre_transfection_donuts')
viewer.add_labels(pre_tf_outer_donuts, name='pre_transfection_outer_donuts')

<Labels layer 'pre_transfection_outer_donuts' at 0x32ba48c50>

In [14]:
# If these pairs of numbers are different, the cell above didn't work properly.
print(image_height, image_width, '|', combined.shape)

1024 1024 | (1024, 1024)


### Remove cells with low transfection (by evaluating mean intensity in the inner donuts against upper and lower intensity thresholds)

In [15]:
# This is purely for visualization, so no need to mess with these parameters too much.
viewer.add_image(
    transfection_img,
    name='transfection',
    colormap='green',         
    contrast_limits=[0, 255], # adjust depending on intensity range
    blending='additive',     
    opacity=0.7 # adjust as needed
)

<Image layer 'transfection' at 0x326c3a510>

In [16]:
transfection_img_norm = (transfection_img - transfection_img.min()) / (transfection_img.max() - transfection_img.min())

# Raise or lower these as needed:
lower_threshold = 0.05  
upper_threshold = 0.2

labels = np.unique(pre_tf_nuclei)
labels = labels[labels != 0]

final_nuclei = np.zeros_like(pre_tf_nuclei)
final_donuts = np.zeros_like(pre_tf_donuts)
final_outer_donuts = np.zeros_like(pre_tf_outer_donuts)

for label_id in labels:
    nucleus_mask = (pre_tf_nuclei == label_id)
    donut_mask = (pre_tf_donuts == label_id)
    outer_mask = (pre_tf_outer_donuts == label_id)

    mean_intensity = transfection_img_norm[donut_mask].mean()

    if mean_intensity >= lower_threshold and mean_intensity <= upper_threshold:
        final_nuclei[nucleus_mask] = label_id
        final_donuts[donut_mask] = label_id
        final_outer_donuts[outer_mask] = label_id

viewer.add_labels(final_nuclei, name='final_nuclei')
viewer.add_labels(final_donuts, name='final_donuts')
viewer.add_labels(final_outer_donuts, name='final_outer_donuts')

<Labels layer 'final_outer_donuts' at 0x326bc7440>

### Evaluate p115 intensity

p115 additions:
- mean intensity, max intensity (brightest pixel), and area for each mask and big mask

Within regions_stats:
Each key is a region. Each value is a list holding the stats of that region, with information in the following order:
- [integer label,
- a ColorArray with the RGBA values that correspond to that integer label,
- {nucleus mask: [mean intensity, max intensity, p115 area in mask]}
- {inner donut mask: [mean intensity, max intensity, p115 area in mask]}
- {outer donut mask: [mean intensity, max intensity, p115 area in mask]}
- {total/regional mask: mean [intensity, max intensity, p115 area in mask]}]

In [17]:
viewer.add_image(
    p115_img,
    name='p115',
    colormap='magma',         
    contrast_limits=[0, 255], # adjust depending on intensity range
    blending='additive',     
    opacity=0.7 # adjust as needed
)

<Image layer 'p115' at 0x33faa9f70>

### Note: Put area values in microns

In [40]:
threshold = 0.05
p115_img_norm = (p115_img - p115_img.min()) / (p115_img.max() - p115_img.min())
regions_stats = {}

layer = viewer.layers['final_nuclei']
labels = np.unique(layer.data)
labels = labels[labels != 0]
label_colors = {label: layer.get_color(label) for label in labels}

for label, color in label_colors.items():
    print(f'Label {label}: {color}')

for label_id in labels:
    nucleus_mask = (final_nuclei == label_id)
    donut_mask = (final_donuts == label_id)
    outer_donut_mask = (final_outer_donuts == label_id)
    regional_mask = nucleus_mask | donut_mask | outer_donut_mask
    
    region_info = {
        'label_color': label_colors[label_id],
        'nucleus_mask_stats': [],
        'donut_mask_stats': [],
        'outer_donut_mask_stats': [],
        'regional_mask_stats': []
    }

    for mask, mask_name in zip([nucleus_mask, donut_mask, outer_donut_mask, regional_mask], 
                               ['nucleus_mask', 'donut_mask', 'outer_donut_mask', 'regional_mask']):
        key = f'{mask_name}_stats'
        p115_overlap_pixels = p115_img_norm[(mask) & (p115_img_norm > 0)]
        region_info[key].append(p115_overlap_pixels.mean() if p115_overlap_pixels.size > 0 else 0)
        region_info[key].append(p115_overlap_pixels.max() if p115_overlap_pixels.size > 0 else 0)
        region_info[key].append((p115_overlap_pixels > threshold).sum())
    
    regions_stats[label_id] = region_info

Label 7: [0.56078434 0.6784314  0.69803923 1.        ]
Label 8: [0.5254902 0.5647059 0.6039216 1.       ]
Label 10: [0.96862745 0.26666668 0.23137255 1.        ]
Label 30: [0.3764706  0.47843137 0.20392157 1.        ]
Label 33: [0.61960787 0.5294118  0.43529412 1.        ]
Label 45: [0.6862745  0.29411766 0.9490196  1.        ]
{np.int32(7): {'label_color': ColorArray([0.56078434, 0.6784314 , 0.69803923, 1.        ], dtype=float32), 'nucleus_mask_stats': [np.float64(0.13609955069673063), np.float64(0.9803921568627451), np.int64(1334)], 'donut_mask_stats': [np.float64(0.18992244296562555), np.float64(1.0), np.int64(2650)], 'outer_donut_mask_stats': [np.float64(0.1500326797385621), np.float64(1.0), np.int64(546)], 'regional_mask_stats': [np.float64(0.16864900821330572), np.float64(1.0), np.int64(4530)]}, np.int32(8): {'label_color': ColorArray([0.5254902, 0.5647059, 0.6039216, 1.       ], dtype=float32), 'nucleus_mask_stats': [np.float64(0.12156699618283254), np.float64(0.847058823529411

### Note: Add description of dataframe below!!

In [43]:
import pandas as pd

regions_df = pd.DataFrame(regions_stats)

regions_df

,7,8,10,30,33,45
label_color,"[0.56078434, 0.6784314, 0.69803923, 1.0]","[0.5254902, 0.5647059, 0.6039216, 1.0]","[0.96862745, 0.26666668, 0.23137255, 1.0]","[0.3764706, 0.47843137, 0.20392157, 1.0]","[0.61960787, 0.5294118, 0.43529412, 1.0]","[0.6862745, 0.29411766, 0.9490196, 1.0]"
nucleus_mask_stats,"[0.13609955069673063, 0.9803921568627451, 1334]","[0.12156699618283254, 0.8470588235294118, 1629]","[0.13784333274685745, 1.0, 1376]","[0.10032325806375499, 1.0, 1570]","[0.15516516951608228, 1.0, 1058]","[0.10860600905522624, 1.0, 1414]"
donut_mask_stats,"[0.18992244296562555, 1.0, 2650]","[0.23246287961094725, 1.0, 2176]","[0.21009297071856806, 1.0, 2479]","[0.21174235439402322, 1.0, 2504]","[0.2011040193203381, 1.0, 1893]","[0.2266368760994902, 1.0, 2197]"
outer_donut_mask_stats,"[0.1500326797385621, 1.0, 546]","[0.2638641680534452, 1.0, 558]","[0.09118994162550514, 0.5450980392156862, 410]","[0.126703146374829, 1.0, 560]","[0.08856486567462299, 0.5882352941176471, 347]","[0.12218844984802431, 1.0, 420]"
regional_mask_stats,"[0.16864900821330572, 1.0, 4530]","[0.19388639437211913, 1.0, 4363]","[0.17323980204056483, 1.0, 4265]","[0.160605437161749, 1.0, 4634]","[0.17258949724394917, 1.0, 3298]","[0.17095259369570612, 1.0, 4031]"


### Note: Add looping for each image set!